## Отчет по лабороторной работе №1

Для начала привожу листинг с некоторыми комментариями

In [ ]:
import string
import math
from itertools import groupby
from itertools import chain
from operator import itemgetter
from numpy import arange

import nltk
from nltk import defaultdict
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

default_tokenize_func = word_tokenize
default_lemmatizer = None # WordNetLemmatizer()
default_stemmer = SnowballStemmer("english")
default_stop_words = set(nltk.corpus.stopwords.words('english'))

# код из eval.py, завернутый в функцию
def eval_metrics(groundtruth_file, answer_file):
    q2reld = {}
    for line in open(groundtruth_file):
        qid, did = [int(x) for x in line.split()]
        if qid in q2reld.keys():
            q2reld[qid].add(did)
        else:
            q2reld[qid] = set()

    q2retrd = {}
    for line in open(answer_file):
        qid, did = [int(x) for x in line.split()]
        if qid in q2retrd.keys():
            q2retrd[qid].append(did)
        else:
            q2retrd[qid] = []

    N = len(q2retrd.keys())
    precision = sum([len(q2reld[q].intersection(q2retrd[q])) * 1.0 / len(q2retrd[q]) for q in q2retrd.keys()]) / N
    recall = sum([len(q2reld[q].intersection(q2retrd[q])) * 1.0 / len(q2reld[q]) for q in q2retrd.keys()]) / N
    fmeasure = 2 * precision * recall / (precision + recall)
    # print("mean precision: {}\nmean recall: {}\nmean F-measure: {}" \
    #       .format(precision, recall, 2 * precision * recall / (precision + recall)))

    # MAP@10
    import numpy as np

    MAP = 0.0
    for q in q2retrd.keys():
        n_results = min(10, len(q2retrd[q]))
        avep = np.zeros(n_results)
        for i in range(n_results):
            avep[i:] += q2retrd[q][i] in q2reld[q]
            avep[i] *= (q2retrd[q][i] in q2reld[q]) / (i + 1.0)
        MAP += sum(avep) / min(n_results, len(q2reld[q]))
    # print("MAP@10: {}".format(MAP / N))
    map10 = MAP / N

    return precision, recall, fmeasure, map10

# функция для парсинга файла
# вырезает куски соответствующие определенной секции в файле и возвращает сам кусок и присвоенный ему идентификатор
def parse_file(file, label='W'):
    section_labels = {'I', 'T', 'A', 'B', 'W'}

    section_found = False
    id = 1

    with open(file, 'r') as f:

        document = ''

        for line in f.readlines():

            stipped_line = line.strip()

            if len(stipped_line) == 0:
                continue

            if stipped_line.startswith('.' + label):
                section_found = True
            elif len(stipped_line) >= 2 and stipped_line[0] == '.' and stipped_line[1] in section_labels:
                if section_found:
                    yield (id, document.strip())
                    document = ''
                    id += 1
                    section_found = False

            elif section_found:
                document += ' ' + stipped_line

        if section_found:
            yield (id, document.strip())

# функция нормализации текста
# бьет текст на токены, удаляет стоп-слова(если передан список слов), применяет лемматизатор и стеммер (переданы)
# возвращает токен с его частотой
def normalize_text(text,
                   tokenize_func=default_tokenize_func,
                   lemmatizer=default_lemmatizer,
                   stemmer=default_stemmer,
                   stop_words=default_stop_words):
    tokens = list()

    for token in tokenize_func(text):
        if token in string.punctuation:
            continue

        if token in stop_words:
            continue

        if lemmatizer:
            token = lemmatizer.lemmatize(token)

        if stemmer:
            token = stemmer.stem(token)

        tokens.append(token)

    tokens.sort()

    return [(key, len(list(group))) for key, group in groupby(tokens)]

# расчет RSV по заданным параметрам (соответствует пункту 3 в задании)
def calc_rsv_default(N, Nt, ftd, ftq, Ld, L, b, k1, k2):
    idf = calc_idf_default(N, Nt)
    tf = ftd * (k1 + 1) / (k1 * ((1 - b) + b * Ld / L) + ftd)
    return idf * tf

# вспомогательная функция для расчета IDF
def calc_idf_default(N, Nt):
    return math.log(1 + (N - Nt + 0.5) / (Nt + 0.5))

# инвертированный индекс с возможностью добавления документов по одному и поиску
class InvertedIndex:
    def __init__(self, lemmatizer=default_lemmatizer, stemmer=default_stemmer):
        self.lemmatizer = lemmatizer
        self.stemmer = stemmer
        self.documents_total_length = 0
        self.documents = dict()
        self.index = defaultdict(list)

        pass

    # добавление документа
    def add_document(self, doc_id, document):
        self.documents_total_length += len(document)
        self.documents[doc_id] = document

        # todo maybe sort after add
        for token in normalize_text(document, lemmatizer=self.lemmatizer, stemmer=self.stemmer):
            self.index[token[0]].append((doc_id, token[1]))

        pass

    # поиск
    def search_okapi_bm25(self, query, b, k1, k2, rsvfunc, norm_rsv, limit=10):
        result = defaultdict(int)

        N = len(self.documents)
        L = self.documents_total_length / len(self.documents)

        idf_sums = defaultdict(int)

        for token_tuple in normalize_text(query, lemmatizer=self.lemmatizer, stemmer=self.stemmer):
            ftq = token_tuple[1]
            token_docs = self.index[token_tuple[0]]
            Nt = len(token_docs)
            for doc_tuple in token_docs:
                doc_id = doc_tuple[0]
                ftd = doc_tuple[1]
                Ld = len(self.documents[doc_id])

                result[doc_id] += rsvfunc(N, Nt, ftd, ftq, Ld, L, b, k1, k2)

                if norm_rsv :
                    idf_sums[doc_id] += calc_idf_default(N, Nt)

        result = list(result.items())

        if norm_rsv:
            result = [(x[0], x[1] / idf_sums[x[0]]) for x in result]

        result = sorted(result, key=itemgetter(1), reverse=True)
        return [x[0] for x in result][:limit]

    # размер словаря
    def get_dist_size(self):
        return len(self.index)
    
    # средняя длина posting-листа
    def get_average_postings_list_length(self):
        l = [len(p) for p in self.index.values()]
        return sum(l) / len(l)

    # максимальная длина posting-листа
    def get_max_postings_list_length(self):
        l = [len(p) for p in self.index.values()]
        return max(l)
    
    def print_statistics(self):
        print("\tStatistics:\n\tDictionary size = {}\n\tAverage postings list length = {}\n\tMax postings list length = {}"
              .format(self.get_dist_size(), self.get_average_postings_list_length(), self.get_max_postings_list_length()))



Заполним инвертированный индекс используя данные тексты.

In [ ]:
# индекс по заголовкам (с использованием стеммера)
indexTS = InvertedIndex()
# индекс по заголовкам (с использованием лемматизатора)
indexTL = InvertedIndex(lemmatizer=WordNetLemmatizer(), stemmer=None)
# индекс по всему abstract (с использованием стеммера)
indexWS = InvertedIndex()
# индекс по всему abstract (с использованием лемматизатора)
indexWL = InvertedIndex(lemmatizer=WordNetLemmatizer(), stemmer=None)
for doc_tuple in parse_file('cran.all.1400', 'T'):
    indexTS.add_document(doc_tuple[0], doc_tuple[1])
    indexTL.add_document(doc_tuple[0], doc_tuple[1])
for doc_tuple in parse_file('cran.all.1400', 'W'):
    indexWS.add_document(doc_tuple[0], doc_tuple[1])
    indexWL.add_document(doc_tuple[0], doc_tuple[1])

Проведем поиск по каждому из индексов и рассчитаем показатели качества (вместе с этим, распечатаем статистику по каждому из индексов)

In [ ]:
def parse_search_eval(index, b=0.75, k1=1.2, k2=None, rsvfunc=calc_rsv_default, norm_rsv=False):
    with open('answer', 'w') as output:
        for query_tuple in parse_file('cran.qry'):
            query_id = query_tuple[0]
            for doc_id in index.search_okapi_bm25(query_tuple[1], b=b, k1=k1, k2=k2, rsvfunc=rsvfunc, norm_rsv=norm_rsv):
                output.write("{} {}\n".format(query_id, doc_id))

    return eval_metrics('qrel_clean', 'answer')

precision, recall, fmeasure, map10 = parse_search_eval(indexTS)
print("Index For Titles (stemmer)\n\tPrecision = {}\n\tRecall = {}\n\tF-measure = {}\n\tMAP@10 = {}".format(precision, recall, fmeasure, map10))
indexTS.print_statistics()

precision, recall, fmeasure, map10 = parse_search_eval(indexTL)
print("Index For Titles (lemmatizer)\n\tPrecision = {}\n\tRecall = {}\n\tF-measure = {}\n\tMAP@10 = {}".format(precision, recall, fmeasure, map10))
indexTL.print_statistics()

precision, recall, fmeasure, map10 = parse_search_eval(indexWS)
print("Index For Abstracts (stemmer)\n\tPrecision = {}\n\tRecall = {}\n\tF-measure = {}\n\tMAP@10 = {}".format(precision, recall, fmeasure, map10))
indexWS.print_statistics()

precision, recall, fmeasure, map10 = parse_search_eval(indexWL)
print("Index For Abstracts (lemmatizer)\n\tPrecision = {}\n\tRecall = {}\n\tF-measure = {}\n\tMAP@10 = {}".format(precision, recall, fmeasure, map10))
indexWL.print_statistics()

Как и ожидалось, все показатели оказались лучше для индекса, построенного по abstracts, чем по titiles, так как увеличивается вероятностью встретить одинаковые термы и в запросе, и в тексте. Если сравнивать индексы, при построении и поиске по которым использовался стеммер с теми, где использовался лемматизатор, видно, что результаты приблизительно одинаковые, поэтому для дальнейших экспериментов решено было использовать стеммер (он быстрее работает). 

In [ ]:
index = indexWS

Далее, используя Grid Search, найдем оптимальные параметры поиска (вычисление занимает несколько минут)

In [ ]:
best_by_fmeasure = (0, 0, 0)
best_by_map10 = (0, 0, 0)

for k1 in arange(1.2, 2.1, .1):
    for b in arange(.0, 1.1, .1):
        precision, recall, fmeasure, map10 = parse_search_eval(index, b=b, k1=k1)
        if fmeasure > best_fmeasure[2]:
            best_by_fmeasure = (precision, recall, fmeasure, map10, b, k1)

        if map10 > best_map10[3]:
            best_by_fmeasure = (precision, recall, fmeasure, map10, b, k1)
            
print("Best f-measure = {} at b = {}, k1 = {}".format(best_by_fmeasure[2], best_by_fmeasure[4], best_by_fmeasure[5]))
print("Best map10 = {} at b = {}, k1 = {}".format(best_by_map10[3], best_by_fmeasure[4], best_by_fmeasure[5]))


!!! Вывод про b и k1

Попробуем заменить функцию вычисления IDF-составляющей, используя при этом значения b и k1, для которых было получено лучшее значение f-меры.

In [ ]:
def calc_rsv_custom_idf(N, Nt, ftd, ftq, Ld, L, b, k1, k2):
    idf = math.log(N / Nt)
    tf = ftd * (k1 + 1) / (k1 * ((1 - b) + b * Ld / L) + ftd)
    return idf * tf

precision, recall, fmeasure, map10 = parse_search_eval(index, b=best_by_fmeasure[4], k1=best_by_fmeasure[5], rsvfunc=calc_rsv_custom_idf)
print("Results:\n\tPrecision = {}\n\tRecall = {}\n\tF-measure = {}\n\tMAP@10 = {}".format(precision, recall, fmeasure, map10))

Как видно, данное изменение не привело к улучшению. Скорее всего потому, что стандартная формула несколько сглаживает используемой значение IDF.

Далее, попробуем нормализовать RSV документа на сумму IDF термов запроса, которые в него входят.

In [ ]:
precision, recall, fmeasure, map10 = parse_search_eval(index, b=best_by_fmeasure[4], k1=best_by_fmeasure[5], norm_rsv=True)
print("Results:\n\tPrecision = {}\n\tRecall = {}\n\tF-measure = {}\n\tMAP@10 = {}".format(precision, recall, fmeasure, map10))

Нормализация несколько ухудшила значение f-меры и сильно ухудшила значение MAP@10. Благодаря такому приему, должно было уменьшиться влияние частовстречающихся слов на полученный RSV, однако, скорее всего из-за того что стоп-слова были отфильтрованы, результаты только ухудшились.